# Hyperparameter exploration

In this notebook, we are going to model the circles dataset again, and see how not only the kernel, but all hyperparameters affect our outcome.

## Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt

X, y = make_circles(n_samples=1000, noise=0.05)

df = pd.DataFrame(dict(x=X[:,0], y=X[:,1], label=y))
colors = {0:'red', 1:'blue'}

fig, ax = plt.subplots()
grouped = df.groupby('label')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])
plt.show()

## MLP

First, we create our training and test sets again:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

Now, let's run the same model with linear kernel as before:

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation

input_dim = X_train.shape[1]
output_dim = 1

model = Sequential()
model.add(Dense(50,input_dim=input_dim))
model.add(Activation('linear'))
model.add(Dense(50))
model.add(Activation('linear'))
model.add(Dense(output_dim))
model.add(Activation('sigmoid'))
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

model.fit(X_train,y_train,epochs=10)

prediction_prob = model.predict(X_test)
prediction_class = (prediction_prob > 0.5).astype("int32")


print('Accuracy:', accuracy_score(y_test,prediction_class))
print('AUC:',roc_auc_score(y_test,prediction_prob))

That's not very good is it? We knew this was coming from playing around in the TensorFlow playground and our earlier experiments. We can visualise the result as well:

In [ ]:
df = pd.DataFrame(dict(x=X_test[:,0], y=X_test[:,1], label=prediction_class[:,0]))
colors = {0:'red', 1:'blue'}

fig, ax = plt.subplots()
grouped = df.groupby('label')
for key, group in grouped:
    group.plot(ax=ax, kind='scatter', x='x', y='y', label=key, color=colors[key])
plt.show()

The linear kernel is clearly not suitable. Let's try something different:

## Different hyperparameters

Let's explore a range of hyperparameters. More specifically, we will focus on:
- Activation function
- Number of neurons in a hidden layer
- Number of layers
- Learning rate
- Number of epochs

First we create the model:

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import Adam

def nn_model(no_neurons,learning_rate,no_layers,kernel):
    model = Sequential()
    model.add(Dense(no_neurons,input_dim=input_dim))
    model.add(Activation(kernel))

    # Extra hidden layers
    for _ in range(0,no_layers):
        model.add(Dense(no_neurons))
        model.add(Activation(kernel))

    # Output
    model.add(Dense(output_dim))
    model.add(Activation('sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),loss='binary_crossentropy',metrics=['accuracy'])
        
    return model

Next, we run the grid search:

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'no_neurons':[50,100],'kernel':['relu','sigmoid','linear'],'no_layers':[1,2],'learning_rate':[0.1,0.01,0.001],'epochs':[10,20],'verbose':[0]} 

grid_search = GridSearchCV(KerasClassifier(nn_model), parameters, cv=5,scoring='roc_auc')
grid_search.fit(X_train, y_train.ravel())

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

print('Mean AUC (+/- standard deviation), for parameters')
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/- %0.03f) for %r"
          % (mean, std, params))

Again, we see that the ReLU kernel clearly outperforms the linear and sigmoid one (although the sigmoid one is slightly higher and above 50%). Besides, we notice that the influence of epochs is apparent: results appear to be much more homogeneous over the various learning rates. The middle learning rate (0.01), however, tends to result in the highest AUC. The structure of the network (number of layers and number of neurons) seems to have little impact on the final results.
This shows how effective hyperparameter search can be in order to finetune your neural network.